# Import

In [ ]:
172.26.64.64



In [ ]:
%%time
import pandas as pd, pyodbc
import datetime as dt; ts = dt.datetime.now(); ts_str = ts.strftime("%Y_%m_%d_%H_%M_%S"); print(ts_str)
con = (r'DRIVER=ODBC Driver 17 for SQL Server;'
       r'SERVER=172.26.64.64;'
       r'DATABASE=MPlanProd;'
       r'Trusted_Connection=no;'
       r'Uid=muserSFT6049;'
       r'Pwd=scgrawmat;')
sql = 'SELECT * FROM [MPlanProd].[dbo].[View_Transaction_SFT_6049]'
cxn = pyodbc.connect(con)
df1 = pd.read_sql(sql, cxn)
df1.to_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', index=False)
df  = df1.copy()
print(df1.shape, ', '.join(df1.columns))

In [ ]:
df.loc[]

In [ ]:
df.TransactionType.value_counts()

In [ ]:
df.SiteName.value_counts()

In [ ]:
df.groupby(['Year']).MillWeight.sum()

In [ ]:
df.groupby(['SiteName','Year']).MillWeight.sum()

# Clean

In [1]:
%%time
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd, numpy as np, datetime as dt, json
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', 999)
ts_str = '2020_09_15_15_18_23'
df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
print(df.shape, ', '.join(df.columns))

(285054, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 1.32 s


In [8]:
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'

In [10]:
df.loc[(df.Sloc=='CH13')&(df.Price_ton==1080)&(df.Month==1)&(df.Year==2020)]['MillWeight'].sum()

142.82999999999998

In [ ]:
print(df.TransactionType.value_counts())

In [ ]:
print(df.TransactionType.value_counts())
df = df.loc[df.TransactionType=='ชั่งซื้อ']
print(df.shape, 1)
df = df.loc[df.SupplierName.notnull()]
print(df.shape, 2)
df = df.loc[df.MillWeight>0]
print(df.shape, 3)
df = df.loc[df.Purchase_price_thb_ton>0]
print(df.shape, 4)
df = df.loc[df.Price_ton>0]
print(df.shape, 5)
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
print(df.shape, 6)
df = df.loc[~df.Sloc.isna()]
print(df.shape, 7)

In [ ]:
df.SiteName.value_counts()

In [ ]:
df.Sloc.value_counts()

In [ ]:
df.WorkCentreName.value_counts()

In [ ]:
df.Sloc.isna().sum()

In [ ]:
df.groupby('Year').MillWeight.sum()

In [ ]:
df_w = df[df.Sloc.str.contains('W1')]
df_w

In [ ]:
df_w.Year.min()

In [ ]:
df_w.MillWeight.sum()

In [ ]:
len(df)

In [ ]:
df.Sloc.str.contains('W1').sum()

In [ ]:
a = df.groupby('Sloc')[['Purchase_price_thb_ton']].sum().merge(df.groupby('Sloc')[['MillWeight']].sum(), left_index=True, right_index=True)
a['mean'] = a['Purchase_price_thb_ton'] / a['MillWeight']

In [ ]:
a

In [ ]:
df.groupby('Sloc')[['MillWeight']].sum()

In [ ]:
df.Purchase_price_thb_ton.sum() / df.MillWeight.sum()

In [ ]:
df.MillWeight.sum()

In [ ]:
df.Purchase_price_thb_ton.sum() / df.MillWeight.sum()

In [ ]:
df = df.loc[~df.Sloc.isin(['W1NS','W1CP','N1BK'])]
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','บางกระทุ่ม','ตาก', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]
print(df.shape)

In [ ]:
ck = ['Zone', 'Sloc','Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton', 'SupplierName']
df = df.drop(df.loc[(df.Year == dt.datetime.now().year) & (df.Month == dt.datetime.now().month)].index)[ck]
df = df.loc[df.Year>=2016].reset_index(drop=True)
print(df.shape) # WorkCentreName, 'Time', 'Ticket_no', SiteName
df.Year = df.Year.astype('str') # Change year type to string
year = dt.datetime.now().year
years = [str(year) for year in range(year-4,year+1)]

# Supply

In [ ]:
supply = df.copy()
supply = supply.groupby(['Year','Month','Price_ton']).agg({'MillWeight':'sum', 'Purchase_price_thb_ton':'sum', 'SupplierName':'nunique', 'Day':'count'}).reset_index()
supply = supply.rename(columns={'Day':'Num_Tran'}).sort_values(by=['Year','Month','Price_ton']).reset_index(drop=True) # , ascending=False

In [ ]:
supply_= supply.copy()
supply = {}
for y in years:
    supply[y] = {}
    for m in range(1,13):
        if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
            supply[y][m] = supply_.loc[(supply_.Year==y)&(supply_.Month==m)]
            supply[y][m] = supply[y][m].merge(supply[y][m][['MillWeight', 'Purchase_price_thb_ton']].cumsum(), left_index=True, right_index=True, suffixes=('','_c'))

In [ ]:
month_valu, month_quan, month_price = {}, {}, {}
for y in years:
    month_valu[y], month_quan[y], month_price[y] = {}, {}, {}
    for m in range(1,13):
        if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
            month_valu[y][m] = supply[y][m].loc[(supply[y][m].Year==y)&(supply[y][m].Month==m)].Purchase_price_thb_ton.sum()
            month_quan[y][m] = supply[y][m].loc[(supply[y][m].Year==y)&(supply[y][m].Month==m)].MillWeight.sum()
            month_price[y][m]= month_valu[y][m]/month_quan[y][m]
            print(f'{y}-{m}, month_valu={month_valu[y][m]:,.2f}, month_quan={month_quan[y][m]:,.2f}, month_price={month_price[y][m]:,.2f}')

In [ ]:
xx, yy, width, value = {}, {}, {}, {}
for y in years:
    xx[y], yy[y], width[y], value[y] = {}, {}, {}, {}
    for m in range(1,13):
        if not ((y==str(dt.datetime.now().year)) & (m in range(dt.datetime.now().month,13))):
            xx[y][m] = []
            yy[y][m] = supply[y][m].loc[(supply[y][m].Year==y)&(supply[y][m].Month==m)].Price_ton.tolist()
            width[y][m] = supply[y][m].loc[(supply[y][m].Year==y)&(supply[y][m].Month==m)].MillWeight.tolist()
            value[y][m] = supply[y][m].loc[(supply[y][m].Year==y)&(supply[y][m].Month==m)].Purchase_price_thb_ton.tolist()
            for i in range(len(width[y][m])):
                if i:
                    xx[y][m] = xx[y][m] + [xx[y][m][i-1] + width[y][m][i-1]/2 + width[y][m][i]/2]
                else:
                    xx[y][m] = xx[y][m] + [width[y][m][i]/2]
            print(f'{y}-{m}', len(width[y][m]), len(xx[y][m]))

In [ ]:
sum_value = sum(value['2020'][1])
fig = go.Figure(data=[go.Bar(
    x=xx['2020'][1],
    y=yy['2020'][1],
    width=width['2020'][1],
    text=value['2020'][1],
)])
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.update_xaxes(dtick=1000)
fig.update_layout(title=f'Total Value: THB {sum_value:,.2f}')
fig.layout.title.font=dict(size=14)
fig.show()

# Cumulative

In [ ]:
months = [dt.date(dt.datetime.now().year, i, 1).strftime('%b') for i in range(1, 13)]
emy = pd.DataFrame()
for y in range(2016, dt.datetime.now().year + 1, 1):
    em = pd.DataFrame(months, columns=['Month'])
    em['Year'] = str(y)
    em['Monthi'] = range(1,13)
    emy = emy.append(em)
emy = emy.reset_index(drop=True)
for m in range(dt.datetime.now().month,13):
    i = emy.loc[(emy.Year==str(dt.datetime.now().year))&(emy.Month==months[m-1])].index
    emy.drop(i, inplace=True)

In [ ]:
ym, ymi, ymt, ymc, ymp = {}, {}, {}, {}, {}; list_all=['all']+list(zone.keys())+list(sloc.keys())
for s in list_all:
    if s == 'all':
        dfs = df.copy()
    elif len(s)==2:
        dfs = df.loc[df.Zone==s].copy()
    else:
        dfs = df.loc[df.Sloc==s].copy()

    ym[s], ymt[s], ymi[s], ymc[s] = {}, {}, {}, {}
    for t in ['Purchase_price_thb_ton','MillWeight']:
        tcount   = dfs.groupby(['Year','Month'])[t].count().reset_index()
        tmean    = dfs.groupby(['Year','Month'])[t].mean().reset_index()
        tmerge   = tcount.merge(tmean, left_index=True, right_index=True, suffixes=('','_m'))
        tmerge   = tmerge.rename(columns={t+'_m':'mean', t:'count'})[['mean','count']]
        
        ym[s][t] = dfs.groupby(['Year','Month'])[t].sum().reset_index()
        ymt[s][t]= ym[s][t].merge(tmerge, left_index=True, right_index=True, suffixes=('_cm',''))
        ymt[s][t]['Monthi'] = ymt[s][t]['Month']
        ymt[s][t]['Month']  = ymt[s][t]['Month'].apply(lambda x: months[int(x)-1])
        ymi[s][t]= ymt[s][t].merge(emy,left_on=['Year','Month','Monthi'],right_on=['Year','Month','Monthi'],how='outer') #emy
        ymi[s][t]= ymi[s][t].sort_values(by=['Year','Monthi']).reset_index(drop=True).fillna(0)
        ymc[s][t]= ymi[s][t].groupby('Year').cumsum().merge(ymi[s][t], left_index=True,right_index=True,suffixes=('_c',''))
        ymc[s][t].drop(columns=['Monthi_c', 'mean_c', 'count_c'], inplace=True)
    
    ymp[s] = ymc[s]['Purchase_price_thb_ton'].merge(ymc[s]['MillWeight'],
                                                    left_on=['Year', 'Month', 'Monthi'],
                                                    right_on=['Year', 'Month', 'Monthi'])
    ymp[s]['Price']  = (ymp[s]['Purchase_price_thb_ton'] / ymp[s]['MillWeight']).fillna(0)
    ymp[s]['Price_c']= (ymp[s]['Purchase_price_thb_ton_c'] / ymp[s]['MillWeight_c']).fillna(0)

In [ ]:
s='all'
for col in ['MillWeight', 'MillWeight_c', 'Price', 'Price_c', 'Purchase_price_thb_ton', 'Purchase_price_thb_ton_c']:
    fig = go.Figure()
    for year in ymp[s].Year.unique():
        Y  = ymp[s].loc[ymp[s].Year==year].copy()
        fig.add_trace(go.Scatter(x=Y['Month'], y=Y[col],
                                 mode="markers+text+lines", textposition="middle right",
                                 text=['' for i in range(len(Y)-1)]+[str(year)]))
    fig.update_layout(showlegend=False, title=col)
    fig.show()

# Models

In [ ]:
%%time
from sktime.forecasting.arima import AutoARIMA
for s in list_all:
    f, forecaster = {}, {}
    for m in months:
        print(s, m)
        
        # 2nd-Step: PQ-Regression
        f[m] = RandomForestRegressor()
        f[m].fit(ymp[s].loc[ymp[s].Month == m].MillWeight.values.reshape(-1, 1),
                 ymp[s].loc[ymp[s].Month == m]['Price'].values)
        # 1st-Step: TS-Forecasting
        hist = ymp[s].loc[ymp[s].Month == m].sort_values(by='Year')
        hist = pd.Series(hist['MillWeight'].astype(float).values)
        forecaster[m] = AutoARIMA()#ThetaForecaster(sp=1)
        forecaster[m].fit(hist)
        yp = forecaster[m].predict([1]).values[0]

        if m in ymp[s].loc[ymp[s].Year == str(dt.datetime.now().year), 'Month'].values:
            ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                              'Year': str(dt.datetime.now().year + 1) + 'F',
                                              'MillWeight': yp}), ignore_index=True)
        else:
            ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                              'Year': str(dt.datetime.now().year) + 'F',
                                              'MillWeight': yp}), ignore_index=True)
            yp = forecaster[m].update_predict_single(pd.Series(yp), fh=[1]).values[0]
            ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                              'Year': str(dt.datetime.now().year + 1) + 'F',
                                              'MillWeight': yp}), ignore_index=True)

            x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F') &
                                (ymp[s].Month == m), 'MillWeight'].values[0]
            y_pred = f[m].predict(x_pred.reshape(1, -1))
            ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F') &
                       (ymp[s].Month == m), 'Price'] = y_pred

            for cm in ['mean_x', 'mean_y', 'count_x', 'count_y']:
                ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F') &
                           (ymp[s].Month == m), cm] = ymp[s].loc[ymp[s].Month == m, cm].mean()

        x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F') &
                            (ymp[s].Month == m), 'MillWeight'].values[0]
        y_pred = f[m].predict(x_pred.reshape(1, -1))
        ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F') &
                   (ymp[s].Month == m), 'Price'] = y_pred

        for cm in ['mean_x', 'mean_y', 'count_x', 'count_y']:
            ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F') &
                       (ymp[s].Month == m), cm] = ymp[s].loc[ymp[s].Month == m, cm].mean()

In [ ]:
ympd = {}
for s in list_all:
    a = ymp[s].copy()
    a['Y'] = 'Year'
    a['YP'] = a['Year'].str.replace('F', '').astype(int) - 2000
    a['YC'] = a['YP']
    a['Type'] = '2016_2019_Actual'
    a.loc[a['Year'] == '2020', 'Type'] = '2020_Actual'
    a.loc[a['Year'] == '2020F', 'Type'] = '2020_Forecast'
    a.loc[a['Year'] == '2021F', 'Type'] = '2021_Forecast'
    b = ymp[s].copy()
    b['Y'] = 'Price'
    b['YP'] = b['Price']
    b['YC'] = b['Year'].str.replace('F', '').astype(int)
    b['Type'] = '2016_2019_Actual'
    b.loc[b['Year'] == '2020', 'Type'] = '2020_Actual'
    b.loc[b['Year'] == '2020F', 'Type'] = '2020_Forecast'
    b.loc[b['Year'] == '2021F', 'Type'] = '2021_Forecast'
    ympd[s] = b.append(a, ignore_index=True)
    ympd[s] = ympd[s].to_json(orient='records') # YearMonthPriceDup

# Export

In [ ]:
wp = {}
for s in sloc.keys():
    df_ = df.loc[df.Sloc==s].groupby(['Year', 'Month', 'Day']).sum().reset_index()
    df_['Price_Daily'] = df_['Purchase_price_thb_ton']/df_['MillWeight']
    df_.drop('Price_ton', axis=1 ,inplace=True)
    wp[s] = df_.to_json(orient = 'records')

In [ ]:
with open('Sloc.json', 'w') as o: json.dump(sloc, o)
with open('Zone.json', 'w') as o: json.dump(zone, o)
with open('wp.json',   'w') as o: json.dump(wp, o)
with open('ympd.json', 'w') as o: json.dump(ympd, o)
with open('list_all.json', 'w') as o: json.dump(list_all, o)
with open('list_all.json', 'r') as o: list_all = json.load(o)
#list_all

In [ ]:
with open('ympd.json', 'r') as o:
    ympd = json.load(o)
for s in list_all:
    ympd[s] = pd.read_json(ympd[s], orient='records')

# Test

# Wood Demand / Supply Model

In [ ]:
def wdsm(s):
    S = ympd[s].loc[ympd[s].Y == 'Year']
    fig1 = px.scatter(S, x='MillWeight', y='YP', color='YC', size='count_y', symbol='Type',
                      color_continuous_scale='Blugrn', facet_col_spacing=0.005, facet_col='Month', height=240)
    fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig1.for_each_yaxis(lambda a: a.update(dtick=1))
    fig1.for_each_yaxis(lambda a: a.update(title='', tickangle=90, dtick=1), row=1, col=1)
    fig1.for_each_xaxis(lambda a: a.update(title='', showticklabels=False))  # showgrid=True, visible=False, dtick=10000
    fig1.layout.update(coloraxis_showscale=False, showlegend=False, margin=dict(l=0, r=0, t=40, b=0)) #, title_font=dict(size=14))
    fig1.update_yaxes(autorange="reversed")  # matches=None

    S = ympd[s].loc[ympd[s].Y == 'Price']
    fig2 = px.scatter(S, x='MillWeight', y='YP', color='YC', size='mean_y', symbol='Type',
                      color_continuous_scale='Blugrn', facet_col_spacing=0.005, facet_col='Month', height=500,
                      labels={'Type': ''})  # , facet_row='Y'
    fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig2.for_each_yaxis(lambda a: a.update(title='', tickangle=90), row=1, col=1)
    fig2.for_each_xaxis(lambda a: a.update(title='')) # , dtick=10000
    fig2.layout.update(coloraxis_showscale=False, legend_orientation='h', margin=dict(l=0, r=0, t=0, b=0))
    t = '''Top: Step_1_Forecast Quantity (Size=Average Transaction Size) |
           Bottom: Step_2_Forecast Price (Size=Number of Transactions) |
           Color: Year (Dasker=Newer)'''
    return t, fig1.show(), fig2.show()

In [ ]:
wdsm('all')

# Sen

In [ ]:
wpm = {}
with open('wp.json', 'r') as o: wp = json.load(o)
for s in sloc.keys():
    wp[s]          = pd.read_json(wp[s], orient = 'records').rename(columns={'Monthi':'Month'})
    wp[s]['dt']    = pd.to_datetime(wp[s][['Year', 'Month', 'Day']])
    #wp[s].drop(['Year', 'Month', 'Day'], axis=1, inplace=True)
    wpm[s]         = wp[s].groupby(['Year','Month']).sum().reset_index().drop('Price_Daily', axis=1)
    wpm[s]['Price_Monthly'] = wpm[s]['Purchase_price_thb_ton']/wpm[s]['MillWeight']
    wpm[s]['Day']  = 1
    wpm[s]['dt']   = pd.to_datetime(wpm[s][['Year', 'Month', 'Day']])
    wpm[s]['Sloc'] = s

In [ ]:
wpms = {}
for s in sloc.keys():
    sen        = wpm[s][['MillWeight','Price_Monthly']].pct_change().apply(lambda x: round(x, 5))*100
    sen['sen'] = sen['MillWeight'] / sen['Price_Monthly']
    sen        = sen.replace(np.inf, 1000).replace(-np.inf, -1000).clip(-100, 100)
    wpms[s]    = sen.merge(wpm[s]['dt'], left_index=True, right_index=True)
    wpms[s]['Sloc'] = s

In [ ]:
wpmj, wpmsj = {}, {};
wpmsjz = pd.DataFrame()
for z in zone.keys():
    a, b = pd.DataFrame(), pd.DataFrame()
    for s in zone[z]:
        a = a.append(wpm[s])
        b = b.append(wpms[s])
    wpmj[z] = a
    wpmsj[z] = b
    wpmsjz = wpmsjz.append(wpmsj[z])

In [ ]:
tl = np.sort(wpmsjz.dt.unique())[::-1]
ss = wpmsjz.Sloc.unique()
dft= pd.DataFrame()
for t in tl:
    dfs = pd.DataFrame(ss, columns=['Sloc'])
    dfs['dt'] = t
    dft = dft.append(dfs)
wpmsjzm = wpmsjz.merge(dft, left_on=['dt','Sloc'], right_on=['dt','Sloc'], how='outer')

In [ ]:
px.line(wpms[s], x='dt', y='sen')